# Zero-shot-classification

`Из предобученной модели достает фичи и ставит им "0"/"1"`

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def extract_features(text):
    features = {
        "is_fever": 0,  # жар
        "is_cough": 0, # кашель
        "is_vomiting": 0, # рвота
        "is_headache": 0, # головная боль
        "is_chest_pain": 0, # боль в груди
        "is_dizziness": 0 # головокружение
    }



    # Список симптомов
    candidate_labels = [elem[3:] for elem in features.keys()]

    result = classifier(text, candidate_labels, multi_label=True)

    for label, score in zip(result['labels'], result['scores']):
        print(f'{label}: {score}')
        if score > 0.5:
            features['is_'+label] = 1

    return features


# text = "Мне 27 лет, живу в Краснодаре, болят руки, температура 38.5, горло болит и жопа, кружится голова"
text = "I am 27 years old, I live in Krasnodar, my hands hurt, the temperature is 42.2, my throat hurts and my ass is dizzy"

# Получение признаков
features = extract_features(text)
print(features)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


dizziness: 0.9744157195091248
fever: 0.8711692094802856
cough: 0.5840808153152466
headache: 0.3607800304889679
chest_pain: 0.012291332706809044
vomiting: 0.0044231475330889225
{'is_fever': 1, 'is_cough': 1, 'is_vomiting': 0, 'is_headache': 0, 'is_chest_pain': 0, 'is_dizziness': 1}


# Бейзлайн

In [ ]:
# классификация на основе ключевых слов
# BoW / TF-IDF + LogReg / SVM

In [172]:
import pandas as pd
data = pd.read_csv('/Users/dlyapin/Documents/git_projects/recsys_diseases/eda/daniil_lyapin_eda_nlp/data/MedDataset.csv')

In [173]:
# через TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(data['symptoms'])
y = data['label']

In [174]:
# pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out()).head()

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y)

LogisticRegression()

In [176]:
X_test_raw = 'Все чешется, покраснения на кожи, боли, сильные зуды, кожа покрывается корочкой, после расчесывания появляется гной.' # Чесотка
X_test = vectorizer.transform([X_test_raw])
pred = model.predict(X_test)
pred

array(['Рак кожи'], dtype=object)

In [177]:
# df_test = pd.DataFrame(X_test.toarray(), columns=vectorizer.get_feature_names_out())
# for col in df_test.columns:
#     if df_test[col].iloc[0] > 0:
#         print(col, df_test[col].iloc[0])

In [178]:
# Через bag of words
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(data['symptoms'])
X_test_raw = 'Все чешется, покраснения на кожи, боли, сильные зуды, кожа покрывается корочкой, после расчесывания появляется гной. ' # Чесотка
X_test = vectorizer.transform([X_test_raw])
pred = model.predict(X_test)
pred

array(['Рак кожи'], dtype=object)

In [ ]:
# score для каждой болезни (топ-5 с наибольшим скором)
pd.concat([data['label'], pd.Series(model.predict_proba(X_test)[0])], axis=1).rename(columns={0:'score'}).sort_values(by='score', ascending=False).head()

,label,score
445,Рак кожи,0.002796
358,Параметрит,0.002563
167,Гонорея,0.002553
480,Стеатогепатит,0.002434
321,Низкое расположение плаценты при беременности,0.002405


### Добавил стемминг

In [ ]:
# добавляю предварительный стемминг (лемматизации нет на русском в nltk!)
df = pd.read_csv('/Users/dlyapin/Documents/git_projects/recsys_diseases/eda/daniil_lyapin_eda_nlp/data/MedDataset_5.csv')
df.head(1)

,target,description,category,reasons,types,symptoms,hards,diagnostics,treatment,profilactics,...,mpw,category_mpw,full_description_len,symptoms_len,bigrams,trigrams,pos_symptoms_freq,pos_symptoms_freq_rus,pos_full_description_freq,pos_full_description_freq_rus
0,Абсцесс Броди,абсцесс брод форм хроническ остеомиелит характ...,болезн опорнодвигательн систем травм,возбудител абсцесс брод штамм стафилококк выяв...,абсцесс могут асептическ септическ развит забо...,абсцесс брод большеберцов кост проявля след си...,абсцесс брод друг форм остеомиелит тяжел патол...,диагностик внутрикостн абсцесс част затрудн вв...,прав лечен заболеван провод стационар терап по...,На сегодняшний день не разработано эффективных...,...,"[('заболеван', 15), ('абсцесс', 12), ('пациент...","['патологическ', 'перелом', 'лечен', 'процесс'...",7193,656,"[('абсцесс', 'брод'), ('брод', 'форм'), ('форм...","[('абсцесс', 'брод', 'форм'), ('брод', 'форм',...","{'NOUN': 32, 'VERB': 12, 'ADJF': 12, 'NPRO': 3...","{'Существительное': 32, 'Глагол': 12, 'Прилага...","{'NOUN': 357, 'VERB': 84, 'ADJF': 120, 'PREP':...","{'Существительное': 357, 'Глагол': 84, 'Прилаг..."


In [130]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
# nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

def preprocess_string(text):
    global stop_words
    text = re.sub(r'\d+', '', text)  # -digits
    text = re.sub(r'[^\w\s]', '', text).replace('  ', ' ')  # -symbols
    text = text.lower()
    words = text.split()
    words = [word for word in words if word not in stop_words]

    stemmer = SnowballStemmer("russian")  # only for rus
    lemmatized_words = [stemmer.stem(word) for word in words]
    lemmatized_words = [word for word in lemmatized_words if len(word) > 3] # hardcode but works
    
    return ' '.join(lemmatized_words)

In [161]:
print('До стемминга: ' + X_test_raw)
X_test_lemm = preprocess_string(X_test_raw)
print('После стемминга: ' + X_test_lemm)

До стемминга: Все чешется, покраснения на кожи, боли, сильные зуды, кожа покрывается корочкой, после расчесывания появляется гной.
После стемминга: чешет покраснен сильн покрыва корочк расчесыван появля


In [169]:
vect = CountVectorizer()
train = vect.fit_transform(df['symptoms'])
test = vect.transform([X_test_lemm])

model = LogisticRegression()
model.fit(train, df['target'])
pred = model.predict(test)
pred

array(['Перелом лучевой кости'], dtype=object)

In [170]:
# смотрю вероятности по всем заболевания
pd.concat([df['target'], pd.Series(model.predict_proba(test)[0])], axis=1).rename(columns={0:'score'}).sort_values(by='score', ascending=False).head()

,target,score
381,Перелом лучевой кости,0.040903
362,Парафимоз,0.027425
101,Воспаление соска,0.025543
51,Ахиллодиния,0.025294
263,Крауроз вульвы,0.024264


In [ ]:
# Что еще стоит попробовать:
# Создание эмбеддингов + обучение простой модели:
# Word2Vec / GloVe + LogReg / Linear Classifier / SVM